In [1]:
import statsmodels.api as sm
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy as sp
import numpy as np
%matplotlib inline
from statsmodels.stats.diagnostic import HetGoldfeldQuandt
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sklearn as sk
from sklearn import datasets
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import model_selection as ms
from sklearn import linear_model as lm
from sklearn.linear_model import LinearRegression
font = {'size'   : 16}
matplotlib.rc('font', **font)

In [2]:
def read_data():
    df = pd.read_csv("data/merged_data.csv")
    df.fillna(0, inplace = True)
    df.drop(set(df[df["HIVdiagnoses"] == 0].index), axis=0, inplace=True)
    cols = ["HIVdiagnoses", "HIVprevalence", "PLHIV", "Population"]
    X = pd.DataFrame(index=df["ADULTMEN"].index, columns=cols)
    for col in cols:
        X[col] = df[col]
    y = np.array(df["HIVincidence"])
    return df, X, y

In [3]:
def read_data_new():
    df = pd.read_csv("data/merged_data.csv")
    df.fillna(0, inplace = True)
    df.drop(set(df[df["HIVdiagnoses"] == 0].index), axis=0, inplace=True)
    cols = ['HIVdiagnoses', 'HIVprevalence', 'MSM12MTH', 'MSM5YEAR']
    X = pd.DataFrame(index=df["ADULTMEN"].index, columns=cols)
    for col in cols:
        X[col] = df[col]
    y = np.array(df["HIVincidence"])
    return df, X, y

In [4]:
hiv_data, X, y = read_data()

In [5]:
hiv_data, Z, y = read_data_new()

In [ ]:
model = sm.OLS(y, Z)
results = model.fit()
results.summary()

In [ ]:
y_logged = np.log(y)

In [ ]:
model = sm.OLS(y_logged, X)
results = model.fit()
results.summary()

In [ ]:
model2 = sm.OLS(y_logged, Z)
results2 = model2.fit()
results2.summary()

In [ ]:
params = results.params
params

In [ ]:
params2 = results2.params
parmas2

In [ ]:
fig, axs = plt.subplots(5,1, figsize=(8,20))

axs[0].scatter(X['HIVdiagnoses'], results.resid)
axs[0].hlines(0,
              X['HIVdiagnoses'].min(), 
              X['HIVdiagnoses'].max(), 
              'k', linestyle='dashed')
axs[0].set_xlabel('HIVdiagnoses')
axs[0].set_ylabel('residuals');

axs[1].scatter(X['HIVprevalence'], results.resid)
axs[1].hlines(0,
              X['HIVprevalence'].min(), 
              X['HIVprevalence'].max(), 
              'k', linestyle='dashed')
axs[1].set_xlabel('HIVprevalence')
axs[1].set_ylabel('residuals');

axs[2].scatter(X['PLHIV'], results.resid)
axs[2].hlines(0,
              X['PLHIV'].min(), 
              X['PLHIV'].max(), 
              'k', linestyle='dashed')
axs[2].set_xlabel('PLHIV')
axs[2].set_ylabel('residuals');

axs[3].scatter(X['Population'], results.resid)
axs[3].hlines(0,
              X['Population'].min(), 
              X['Population'].max(), 
              'k', linestyle='dashed')
axs[3].set_xlabel('Population')
axs[3].set_ylabel('residuals');


axs[4].scatter(results.fittedvalues, results.resid)
axs[4].hlines(0,
              results.fittedvalues.min(), 
              results.fittedvalues.max(),
              'k', linestyle='dashed')
axs[4].set_xlabel('predicted mpg')
axs[4].set_ylabel('residuals');

In [ ]:
fig, axs = plt.subplots(5,1, figsize=(8,20))

axs[0].scatter(Z['HIVdiagnoses'], results2.resid)
axs[0].hlines(0,
              Z['HIVdiagnoses'].min(), 
              Z['HIVdiagnoses'].max(), 
              'k', linestyle='dashed')
axs[0].set_xlabel('HIVdiagnoses')
axs[0].set_ylabel('residuals');

axs[1].scatter(Z['HIVprevalence'], results2.resid)
axs[1].hlines(0,
              Z['HIVprevalence'].min(), 
              Z['HIVprevalence'].max(), 
              'k', linestyle='dashed')
axs[1].set_xlabel('HIVprevalence')
axs[1].set_ylabel('residuals');

axs[2].scatter(Z['MSM12MTH'], results2.resid)
axs[2].hlines(0,
              Z['MSM12MTH'].min(), 
              Z['MSM12MTH'].max(), 
              'k', linestyle='dashed')
axs[2].set_xlabel('MSM12MTH')
axs[2].set_ylabel('residuals');

axs[3].scatter(Z['MSM5YEAR'], results2.resid)
axs[3].hlines(0,
              Z['MSM5YEAR'].min(), 
              Z['MSM5YEAR'].max(), 
              'k', linestyle='dashed')
axs[3].set_xlabel('MSM5YEAR')
axs[3].set_ylabel('residuals');


axs[4].scatter(results2.fittedvalues, results2.resid)
axs[4].hlines(0,
              results2.fittedvalues.min(), 
              results2.fittedvalues.max(),
              'k', linestyle='dashed')
axs[4].set_xlabel('predicted mpg')
axs[4].set_ylabel('residuals');

In [ ]:
f_statistic, p_value, _ = sm.stats.diagnostic.het_goldfeldquandt(y_logged, X, idx=1, alternative='two-sided')
print(p_value)

In [ ]:
f_statistic2, p_value2, _ = sm.stats.diagnostic.het_goldfeldquandt(y_logged, Z, idx=1, alternative='two-sided')
print(p_value2)

In [ ]:
stud_resids = results.outlier_test()['student_resid']


In [ ]:
stud_resids2 = results2.outlier_test()['student_resid']

In [ ]:
ax = sm.graphics.qqplot(stud_resids, line='45')

In [ ]:
X_train, X_test, y_train, y_test = ms.train_test_split(X.values, y_logged, test_size =.25)
model = sm.OLS(y_train, X_train)
res = model.fit()
y_pred = res.predict(X_test)

In [ ]:
r = sp.stats.linregress(y_pred,y_test)
r[2]

In [ ]:
X_train2, X_test2, y_train2, y_test2 = ms.train_test_split(Z.values, y_logged, test_size =.25)
model2 = sm.OLS(y_train2, X_train2)
res2 = model.fit()
y_pred2 = res.predict(X_test2)
r2 = sp.stats.linregress(y_pred2,y_test2)
r2[2]

In [ ]:
reg = LinearRegression(fit_intercept=True).fit(X, y_logged)

In [ ]:
cvr = ms.cross_validate(reg,X,y,cv=20,return_train_score=True)

In [ ]:
print(cvr)

In [ ]:
reg2 = LinearRegression(fit_intercept=True).fit(Z, y_logged)
cvr2 = ms.cross_validate(reg,Z,y,cv=20,return_train_score=True)
print(cvr2)

In [ ]:
from yellowbrick.regressor import ResidualsPlot

In [ ]:
visualizer = ResidualsPlot(reg)
visualizer.fit(X_train, y_train)
visualizer.score(X_test, y_test)
visualizer.poof() 

In [ ]:
visualizer2 = ResidualsPlot(reg2)
visualizer2.fit(X_train2, y_train2)
visualizer2.score(X_test2, y_test2)
visualizer2.poof() 